In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
from sklearn.model_selection import train_test_split



In [2]:
stock1 = pd.read_csv('stock_1.csv')
stock1

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.000754,1.001542,1.000689,1.001607,1,25,25,100,1
1,5,1,1.000754,1.001673,1.000689,1.001739,26,60,25,100,1
2,5,2,1.000754,1.001411,1.000623,1.001476,1,25,25,125,1
3,5,3,1.000754,1.001542,1.000689,1.001607,125,25,126,36,1
4,5,4,1.000754,1.001476,1.000623,1.001542,100,100,25,25,1
...,...,...,...,...,...,...,...,...,...,...,...
1507527,32767,588,0.998911,0.999109,0.998812,0.999208,126,42,101,100,1
1507528,32767,589,0.998911,0.999109,0.998812,0.999208,126,126,101,200,1
1507529,32767,591,0.998911,0.999109,0.998812,0.999208,126,226,101,200,1
1507530,32767,592,0.998911,0.999109,0.998812,0.999208,226,225,101,100,1


In [4]:
# Cecilia & Callum Code
# Code to calculate Weighted Average Price for a Stock at a time.
stock1["WAP"] = (stock1["bid_price1"] * stock1["ask_size1"] + stock1["ask_price1"] * stock1["bid_size1"]) / (stock1["bid_size1"] + stock1["ask_size1"])
stock1



,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,WAP
0,5,0,1.000754,1.001542,1.000689,1.001607,1,25,25,100,1,1.000785
1,5,1,1.000754,1.001673,1.000689,1.001739,26,60,25,100,1,1.001032
2,5,2,1.000754,1.001411,1.000623,1.001476,1,25,25,125,1,1.000780
3,5,3,1.000754,1.001542,1.000689,1.001607,125,25,126,36,1,1.001411
4,5,4,1.000754,1.001476,1.000623,1.001542,100,100,25,25,1,1.001115
...,...,...,...,...,...,...,...,...,...,...,...,...
1507527,32767,588,0.998911,0.999109,0.998812,0.999208,126,42,101,100,1,0.999060
1507528,32767,589,0.998911,0.999109,0.998812,0.999208,126,126,101,200,1,0.999010
1507529,32767,591,0.998911,0.999109,0.998812,0.999208,126,226,101,200,1,0.998982
1507530,32767,592,0.998911,0.999109,0.998812,0.999208,226,225,101,100,1,0.999011


In [9]:
# Callum & Cecilia Code

# Creating empty list to store the log returns for each time period.
log_r1 = []
# Getting an array of unique time IDs from teh first column of the dataframe (is this the index assigned automatically?)
time_IDs = np.unique(stock1.iloc[:, 0])

# For every unique time id
for i in range(len(time_IDs)):
    
    # Calculating the seconds in bucket and WAP values for rows where the time ID matches the current ID. 
    sec = stock1.loc[stock1.iloc[:, 0] == time_IDs[i], 'seconds_in_bucket'].values
    price = stock1.loc[stock1.iloc[:, 0] == time_IDs[i], 'WAP'].values

    # Calculating the log returns for this time period
    log_r = np.log(price[1:] / price[0:(len(price) - 1)])
    
    # Adding the log returns for this time period to the list. 
    log_r1.append(pd.DataFrame({'time': sec[1:], 'log_return': log_r}))
    # Check if any time periods were skipped.
    time_no_change = np.setdiff1d(np.arange(1, 601), log_r1[i]['time'].values)
    # Anytime periods that were skipped set them to have a log return of 0
    if len(time_no_change) > 0:
        new_df = pd.DataFrame({'time': time_no_change, 'log_return': 0})
        log_r1[i] = pd.concat([log_r1[i], new_df])
        log_r1[i] = log_r1[i].sort_values(by='time')

# List to store the volatility values for each time period. 
vol = []

# Function to compute the volatility from the log returns. 
def comp_vol(x):
    return np.sqrt(np.sum(x ** 2))

# For every time period
for i in range(len(log_r1)):
    # We assign each log return to a time bucket (group of 30 seconds)
    log_r1[i]['time_bucket'] = np.ceil(log_r1[i]['time'] / 30)
    # Compute the volatility for each time bucket using the comp_vol function.
    vol.append(log_r1[i].groupby('time_bucket')['log_return'].agg(comp_vol).reset_index())
    vol[i].columns = ['time_bucket', 'volatility']
    

In [15]:
print(len(vol))
print(len(log_r1))
print(stock1['time_id'].nunique())

3830
3830
3830


In [32]:
import sklearn

# Initialising empty list
ans = []
# Looping through every element in the volatility list
for i in range(len(vol)):
    for j in range(20):
        ans.append(vol[i]["volatility"][j])
y = np.array(ans)

train, test = sklearn.model_selection.train_test_split(y, train_size=8000, shuffle=False)

In [33]:
print(len(train))
print(len(test))

8000
68600


In [36]:
# GARCH MODEL
model = arch_model(train, vol='GARCH', p=1, q=1)
results = model.fit()

prediction = results.forecast(horizon=len(test), method='simulation')
test['predicted_volatility'] = np.sqrt(prediction.variance.values[-1, :])



/opt/homebrew/lib/python3.9/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 7.952e-07. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/lib/python3.9/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/opt/homebrew/lib/python3.9/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ impor

Iteration:      1,   Func. Count:      5,   Neg. LLF: -47448.82125558854
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -47448.82124633211
            Iterations: 1
            Function evaluations: 5
            Gradient evaluations: 1


KeyboardInterrupt: 

In [ ]:
# Model performance evaluation
rmse = np.sqrt(((test['volatility'] - test['predicted_volatility'])**2).mean())
print(f'RMSE: {rmse}')